# Projet 7 :  Détectez les Bad Buzz grâce au Deep Learning
## Website
<font color = "grey"><i> Chargement des modules necessaires </i></font>

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np

import pandas as pd
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import spacy
import re
import string
import os

import dash_bootstrap_components as dbc
from dash.dependencies import Output, State, Input

C:\Users\marj1\AppData\Local\Temp/ipykernel_37040/888921169.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\marj1\AppData\Local\Temp/ipykernel_37040/888921169.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


<font color = "grey"><i> Chargement du dataframe  </i></font>

In [2]:
import plotly.graph_objects as go





import main

assets_path = os.getcwd() +'\data'
print("path :",assets_path)

path : C:\Users\marj1\PycharmProjects\projet7\data


In [3]:
def load_model():
    global RNN
    global tokenizer
    global nlp
    print('chargement')
    RNN = tensorflow.keras.models.load_model('saved_model/', compile=True)

    data_df = pd.read_csv('data/under_sampling_10000_clean.csv',
                          encoding='latin',
                          delimiter=",")
    data_df['clean_tweet'] = data_df['clean_tweet'].astype(str)
    tokenizer = Tokenizer(nb_words=2500, lower=True, split=' ')
    tokenizer.fit_on_texts(data_df['clean_tweet'].values)
    nlp = spacy.load('en_core_web_sm')



def preprocess_reviews(reviews):
    REPLACE_NO_SPACE = re.compile("[.;:!\?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|(\n)|(')|[0-9]")
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews


def get_lemmatized_text(corpus):
    lem = [' '.join([token.lemma_ for token in nlp(review)]) for review in corpus]
    return lem



def remove_stop_words(corpus, stop_words):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split()
                      if word not in stop_words])
            )
    return removed_stop_words

def predict(tweet):

    MAX_WORDS = 21

    out = [tweet]
    out = preprocess_reviews(out)

    sp = string.punctuation
    out = list(map(lambda t: ''.join(["" if c.isdigit() else c for c in t]), out))
    out = list(map(lambda t: ''.join(["" if c in sp else c for c in t]), out))
    out = list(map(str.lower, out))

    out = get_lemmatized_text(out)
    stop_words_ot = ['a', 'about', 'above', 'the', 'after', 'my', 'me', 'our',
                     'your', 'i', 'we', 'you', 'he', 'she', 'they',
                     'their', 'in', 'until', 'before', 'it', 'and', 'at', 'of']

    out = remove_stop_words(out, stop_words_ot)

    X = tokenizer.texts_to_sequences(out)

    vect = pad_sequences(X, maxlen=MAX_WORDS, padding='post')

    sentiment = RNN.predict(vect)

    my_prediction = np.argmax(sentiment, axis=1)

    result = ['positif' if lbl == 1 else 'négatif' for lbl in my_prediction]

    return(result)

<font color = "grey"><i> Configuration donnée dashboard </i></font>

In [4]:
load_model()

chargement


c:\users\marj1\pycharmprojects\projet7\env\lib\site-packages\keras_preprocessing\text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [5]:
x = predict("I feel good")
print("I feel good :",x)

['positif']


In [6]:
x = predict("I feel bad")
print("I feel bad:", x)



['négatif']


In [7]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css',dbc.themes.BOOTSTRAP]
load_model()
app = dash.Dash(__name__,assets_folder=assets_path,
                external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
             html.Div([
                html.H1("Prédiction de sentiment",style={'font-size': '60px'})
                  ], className='row'),
                html.Div(children=[
                    html.Div(children=[
                         dcc.Input(id='username', value='I feel bad', type='text', style={'font-size': '20px'})
                    ], className='two columns',style = {'width': '25%','height': '100%'}),
                    html.Div(children=[
                          html.Button(id='submit-button', type='submit', children='Prédire',
                                      className='button-primary',style = {'font-size': '20px','border-radius': '8px'}),
                    ], className='two columns',style = {'padding-rigth': '2%'}),
                html.Div(children=[
                    dbc.Alert(id='output_div', color="primary",style={'font-size': '20px'})
                ], className='row',style = {'padding': '2%'}),



                    ], className='row')
    ],style = {'padding' : '20%'})

@app.callback(Output('output_div', 'children'),
              [Input('submit-button', 'n_clicks')],
              [State('username', 'value')],
              )
def update_output(clicks, input_value):
    if clicks is not None:
        print(clicks, input_value)
        output = predict(input_value)
        output = str(output[0])
        print(clicks, input_value, ' pred',output)
        return output

chargement


<font color = "grey"><i> Lancement dashboard  </i></font>

In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
